In [2]:
import pandas as pd
POSSIBLE_LABELS = 'yes no up down left right on off stop go silence unknown'.split()
id2name = {i: name for i, name in enumerate(POSSIBLE_LABELS)}


In [31]:
df_trans = pd.read_csv('./speech-api/transcription.csv')
df_trans.columns = ['fname', 'trans_label']
df_submission = pd.read_csv('./submissions/submission_083_merged4.csv')

In [32]:
df_res = df_trans.merge(df_submission, on = 'fname', how = 'outer')

In [45]:
df_corr = df_res.loc[df_res['trans_label'].isin(POSSIBLE_LABELS)]
df_corr = df_corr.loc[df_corr['label'] != df_corr['trans_label']]
df_corr.columns = ['fname', 'new_label', 'drop']
del df_corr['drop']
df_corr.shape

(43, 2)

In [58]:
df_sub_corrected = df_corr.merge(df_submission, on = 'fname', how = 'outer')
df_sub_corrected = df_sub_corrected.fillna('')

In [61]:
import numpy as np
df_sub_corrected['final_label'] = np.where(df_sub_corrected['new_label'] == '', df_sub_corrected['label'], df_sub_corrected['new_label'])

In [64]:
df_sub_corrected = df_sub_corrected[['fname', 'final_label']]
df_sub_corrected.columns = ['fname', 'label']

In [67]:
df_sub_corrected.to_csv('./submissions/new/083_corrected_5k.csv', index=False)

In [3]:
df_dup = pd.read_csv('./speech-api/duplicates-Kaggle-Speech-Recognition-Inventory.csv')

FileNotFoundError: File b'./speech-api/duplicates-Kaggle-Speech-Recognition-Inventory.csv' does not exist

In [71]:
df_dup[df_dup['filename'] == 'test/audio/clip_0064f7bae.wav']

,filename,dataset,word,speakerID,utteranceIndex,extension,samples,min,Q10,Q25,median,Q75,Q90,max,range,mean,sd,md5sum
1911,test/audio/clip_0064f7bae.wav,test,NaN,0064f7bae,0,wav,16000,0,0.0,0.0,0.0,0.0,0.0,0,0,0.0,0.0,800ea34119b791a1be054aafd09fc896


In [73]:
df_silence = df_dup.loc[df_dup['min'] == 0]

In [76]:
df_silence = df_silence[['filename', 'word']]

In [87]:
df_silence = df_silence[1:]
df_silence['label'] = 'silence'
df_silence['fname'] = df_silence['filename'].apply( lambda filename: filename.split('/')[2])

In [90]:
df_silence = df_silence[['fname', 'label']]
df_silence.to_csv('./speech-api/silence_test.csv', index=False)

In [93]:
df_sub_corrected_silence = df_silence.merge(df_submission, on = 'fname', how = 'inner')

In [96]:
np.sum(df_sub_corrected_silence['label_x'] == df_sub_corrected_silence['label_y'])

3701

In [97]:
df_sub_corrected_silence.shape

(3701, 3)